# Build kinetic models from the tfa model and sampling

In [1]:
# Load the tfa model 
from pytfa.io.json import load_json_model
model_file = 'reduced_model_ETC_core_20240816-155234_continuous.json'
tmodel = load_json_model(model_file)
sol = tmodel.optimize()

2024-12-13 15:46:00,126 - thermomodel_Recon3thermoCurated - continuous - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# Import the a kinetic model 
from skimpy.io.generate_from_pytfa import FromPyTFA

small_molecules = ['h_c', 'h_e','h_i', 'h_m', 'na1_e', 'na1_c', 'na1_m','k_e','k_c']

model_gen = FromPyTFA(small_molecules=small_molecules, max_revesible_deltag_0=1e3,)

kmodel = model_gen.import_model(tmodel, sol.raw)

kmodel.name = "muscle_kinetic_model"


2024-12-13 15:46:00,673 - Unnamed - WARNING - Metabolite bhb_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,674 - Unnamed - WARNING - Metabolite hdca_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,674 - Unnamed - WARNING - Metabolite co2_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,675 - Unnamed - WARNING - Metabolite h_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,676 - Unnamed - WARNING - Metabolite lac_L_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,677 - Unnamed - WARNING - Metabolite o2_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024-12-13 15:46:00,677 - Unnamed - WARNING - Metabolite glc_D_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2024

In [3]:
# Impose boundary conditions

from skimpy.core.modifiers import ConstantConcentration

for r in tmodel.boundary:
    met = [m for m in r.metabolites][0]
    try:
        k_met = kmodel.reactants[met.id]
        boundary = ConstantConcentration(k_met)
        kmodel.add_boundary_condition(boundary)
        print(k_met)
    except:
        print(f'met {met} not in model')

# Also assume intracellular sodium to be constant
# boundary = ConstantConcentration(kmodel.reactants['na1_c']) 
# kmodel.add_boundary_condition(boundary)

bhb_e
hdca_e
co2_e
h_e
met h2o_e not in model
lac_L_e
o2_e
glc_D_e
pyr_e


In [4]:
# Export the model to yaml
from skimpy.io.yaml import export_to_yaml
export_to_yaml(kmodel, model_file.replace("_continuous.json", "_kinetic.yml") )


True

In [5]:
1/0

ZeroDivisionError: division by zero

# Parameterize the kinetic model following ORACLE framework

In [ ]:
# Reload and prepare the model
from skimpy.io.yaml import load_yaml_model
kmodel = load_yaml_model(model_file.replace("_continuous.json", "_kinetic_curated.yml"))

# Compile the jacobian expressions
NCPU = 12
kmodel.prepare()
kmodel.compile_jacobian(ncpu=NCPU)

In [ ]:
# Initiate a parameter sampler
from skimpy.sampling.simple_parameter_sampler import SimpleParameterSampler

params = SimpleParameterSampler.Parameters(n_samples = 100)
sampler = SimpleParameterSampler(params)

In [ ]:
# Load TFA samples 
import pandas as pd 
tfa_sample_file = 'reduced_model_ETC_core_20240701-184451_tfa_sampling.csv'
tfa_samples = pd.read_csv(tfa_sample_file)

In [ ]:
# Scaling parameters
CONCENTRATION_SCALING = 1e6 # 1 mol to 1 umol
TIME_SCALING = 1 # 1min
DENSITY = 1200 # g/L 
GDW_GWW_RATIO = 1.0 # Fluxes are in gWW



In [ ]:
from skimpy.analysis.oracle.load_pytfa_solution import load_fluxes, load_concentrations,\
      load_equilibrium_constants
from skimpy.core.parameters import ParameterValuePopulation

lambda_max_all = []
lambda_min_all = []

# Make a new directory for the output
import os
os.makedirs(tfa_sample_file.replace("csv",''), exist_ok=True)

path_for_output = './'+tfa_sample_file.replace("csv",'')+'/paramter_pop_{}.h5'


for i, sample in tfa_samples.iterrows():
    # Load fluxes and concentrations
    fluxes = load_fluxes(sample, tmodel, kmodel,
                            density=DENSITY,
                            ratio_gdw_gww=GDW_GWW_RATIO,
                            concentration_scaling=CONCENTRATION_SCALING,
                            time_scaling=TIME_SCALING)
    concentrations = load_concentrations(sample, tmodel, kmodel,
                                            concentration_scaling=CONCENTRATION_SCALING)
    
    # Fetch equilibrium constants
    load_equilibrium_constants(sample, tmodel, kmodel,
                               concentration_scaling=CONCENTRATION_SCALING,
                               in_place=True)
    
    # Generate sampels and fetch slowest and fastest eigenvalues
    params, lamda_max, lamda_min = sampler.sample(kmodel, fluxes, concentrations,
                                                    only_stable=False,
                                                    min_max_eigenvalues=True)
    lambda_max_all.append(pd.DataFrame(lamda_max))
    lambda_min_all.append(pd.DataFrame(lamda_min))

    params_population = ParameterValuePopulation(params, kmodel)
    params_population.save(path_for_output.format(i))


# Process df and save dataframe
lambda_max_all = pd.concat(lambda_max_all, axis=1)
lambda_min_all = pd.concat(lambda_min_all, axis=1)


KeyError: 'AKGMALtm'